In [1]:
!pip install fsspec==2024.10.0 --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `clinicaltrialsnlp` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `

In [3]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda"

In [5]:
import pandas as pd
from transformers import LlamaTokenizer, LlamaForSequenceClassification, Trainer, TrainingArguments
import torch

csv_file = "synthetic_patient_notes_no_sex_age.csv"
df = pd.read_csv(csv_file)

print(df.head())

                                            Trial ID  \
0  This study is a follow-up of the GRECCAR 07 co...   
1  This study is a follow-up of the GRECCAR 07 co...   
2  This clinical trial is a Phase 1-2, open-label...   
3  This clinical trial is a Phase 1-2, open-label...   
4  The objective of this study is to constitute c...   

                                       Patient Notes  Eligible  
0  Patient with confirmed colostomy stoma seeking...         1  
1  A medical history inconsistent with colostomy ...         0  
2  A 42-year-old with a history of solid tumor, a...         1  
3  Patient with chronic conditions that do not al...         0  
4  Patient with confirmed crohn disease seeking t...         1  


In [6]:
pip install datasets

In [7]:
from datasets import Dataset
from transformers import AutoTokenizer

def preprocess_function(examples):
    tokenizer.pad_token = tokenizer.eos_token;
    inputs = [f"Trial: {trial} Patient: {note}" for trial, note in zip(examples["Trial ID"], examples["Patient Notes"])]
    model_inputs = tokenizer(inputs, padding=True, truncation=True, max_length=512)
    model_inputs["labels"] = examples["Eligible"]  # Set labels (1 for eligible, 0 for ineligible)
    return model_inputs

dataset = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

dataset = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [8]:
pip install -U bitsandbytes

In [ ]:
from transformers import AutoModel, TrainingArguments

# training arguments
training_args = TrainingArguments(
    output_dir="./llama-patient-trial-matching",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    gradient_accumulation_steps=2,
)

model = AutoModel.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", num_labels=2)  # Binary classification

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

# save the fine-tuned model
model.save_pretrained("./fine-tuned-llama")
tokenizer.save_pretrained("./fine-tuned-llama")

model_id = "fine-tuned-llama"

model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_int_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModel.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"": 0})


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding(text):
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

patient_description = "Chronic fatigue and muscle pain"

trial_descriptions = [
    "Study on fibromyalgia treatment",
    "Research on chronic pain management",
    "Clinical trial for asthma patients"]

patient_embedding = get_embedding(patient_description)
trial_embeddings = [get_embedding(trial) for trial in trial_descriptions]

similarities = [cosine_similarity(patient_embedding, trial_emb)[0][0] for trial_emb in trial_embeddings]

ranked_trials = sorted(zip(trial_descriptions, similarities), key=lambda x: x[1], reverse=True)
for trial, score in ranked_trials:
    print(f"{trial}: {score:.4f}")

In [ ]:
# Clean and make sense of clinical trial data
import pandas as pd

trial_df = pd.read_csv('ctg-studies.csv')
trial_df.dropna(how='any')
trial_dict = {} # Maps from trial ID to the population it treats

trial_df['Conditions'] = trial_df['Conditions'].fillna('').astype(str).str.lower()
trial_df['Sex'] = trial_df['Sex'].fillna('').astype(str).str.lower()
trial_df['Age'] = trial_df['Age'].fillna('').astype(str).str.lower()

trial_df = trial_df.sample(n=3000, random_state=42)

age_dict = {"child": (0, 18),
            "child, adult": (0, 65),
            "child, adult, older_adult": (0, 100),
            "adult": (18, 65),
            "adult, older_adult": (18, 100),
            "older_adult": (65, 100)}

sex_dict = {"male": 0,
            "female": 1}

for row in range(len(trial_df)):
  sex = 2

  if trial_df.iloc[row]['Sex'] in sex_dict:
    sex = sex_dict[trial_df.iloc[row]['Sex']]

  age = (0, 100)

  if trial_df.iloc[row]['Age'] in age_dict:
    age = age_dict[trial_df.iloc[row]['Age']]

  curr_trial = {"Condition": trial_df.iloc[row]['Conditions'],
                "Age": age,
                "Sex": sex}

  trial_dict[trial_df.iloc[row]['NCT Number']] = curr_trial

print(trial_dict)

In [ ]:
print(len(trial_dict.keys()))

# Cache get_embedding for each trial condition
trial_embedding_dict = {}

count = 1

for trial in trial_dict.keys():
    trial_embedding_dict[trial] = get_embedding(trial_dict[trial]['Condition'])
    print(count)
    count += 1

In [ ]:
import pickle

# Save embeddings to a file
# with open("embeddings.pkl", "wb") as f:
#     pickle.dump(trial_embedding_dict, f)

# Load embeddings from the file
with open("embeddings.pkl", "rb") as f:
    loaded_embeddings = pickle.load(f)
print(type(loaded_embeddings))

In [ ]:
# TODO: MAKE SURE ALL DATA IS IN THE RIGHT FORMAT
import pandas as pd

patient_df = pd.read_csv('patient_population_data.csv')

mrr = 0

def calculate_mrr(patient_condition, ranked_trials):
    # Normalize patient_condition for comparison
    normalized_condition = patient_condition.lower().split()

    index = 1
    for trial_id, score, trial_description in ranked_trials:
        # Normalize the trial condition and check if it contains the target condition
        for token in normalized_condition:
          if token in trial_description.lower():
            # Reciprocal rank is 1 divided by the position in the list
            return 1 / index
        index += 1

    # If the condition is not found in any trial, MRR is 0
    return 0.0

# def match_patient_to_trials(patient_df):
trial_similarities = []

for _, patient_row in patient_df.iterrows():
  patient_description = patient_row['DISEASE_DISORDER']
  patient_embedding = get_embedding(patient_description)

  # for the current patient
  patient_trial_similarities = []

  for trial in loaded_embeddings.keys():
      trial_embedding = loaded_embeddings[trial]
      similarity_score = cosine_similarity(patient_embedding, trial_embedding)[0][0]

      age_match = True
      sex_match = True

      # if the patient's age is provided
      if pd.notna(patient_row['AGE']):
          trial_age_range = trial_dict[trial]["Age"]
          if not (trial_age_range[0] <= patient_row['AGE'] <= trial_age_range[1]):
              age_match = False

      # convert patient's sex
      patient_sex = None
      if pd.notna(patient_row['SEX']):
        if patient_row['SEX'].lower() == 'male' or patient_row['SEX'].lower() == 'man':
          patient_sex = 0
        elif patient_row['SEX'].lower() == 'female' or patient_row['SEX'].lower() == 'woman':
          patient_sex = 1
        else:
          patient_sex = 2

      if pd.notna(patient_row['SEX']):
          if trial_dict[trial]["Sex"] != 2 and trial_dict[trial]["Sex"] != patient_sex:
              sex_match = False

      # if both age and sex match (if provided)
      if age_match and sex_match:
          patient_trial_similarities.append((trial, similarity_score, trial_dict[trial]['Condition']))

  # sort
  patient_trial_similarities.sort(key=lambda x: x[1], reverse=True)

  top_trials = patient_trial_similarities[:10]

  mrr += calculate_mrr(patient_description, top_trials)


In [ ]:
print(mrr / len(patient_df))